# Anki2Roam

### Usage
1. Exit from Anki
1. Find your Anki data folder https://docs.ankiweb.net/#/files?id=file-locations
1. Inside that folder - open a subdirectory that matches your profile name
1. Inside that directory there would be `collection.anki2` file which is where all your decks and notes and stored.
1. Click button `Upload` below and select that files.
1. You should be able to select one of your decks to export.

In [2]:
from pathlib import Path
from ipywidgets import *
import anki2roam
from anki import Collection

# bunch of inspiration from https://github.com/fomightez/3Dscatter_plot-binder/blob/master/3D_scatter_Voila_matplotlibSTREAMLINED.ipynb

#select_markdown = widgets.HTML(value=markdown.markdown("""Use above button to upload your data file if not among the choices you see listed below.<br/>**Choose the file with the data to plot from below**:""")) # based on https://github.com/jupyter-widgets/ipywidgets/issues/2428#issuecomment-500084610 and https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#HTML ; need to pip install markdown first and import it

out = widgets.Output()


auto_layout = lambda: Layout(height='auto', width='auto')
out_text = Textarea(placeholder='Markdown output will appear here', layout=auto_layout())

select_deck = Dropdown(options=[], description='Decks:', layout=auto_layout())

collection_fname = "collection.anki2"

@out.capture()
def show_decks(change):
    out.clear_output()

    filename = next(iter(change['new']))
    Path(collection_fname).write_bytes(change['new'][filename]['content'])

    collection = Collection(collection_fname, log=True)
    select_deck.options=tuple(collection.decks.allNames())
    collection.close()

@out.capture()
def export_deck(button):
    exporter = anki2roam.MarkdownExporter(select_deck.value, ".")
    out_text.value = exporter.export_text()
    print("The deck had the following images: ", exporter.images)

file_uploader = FileUpload(layout=auto_layout())
file_uploader.observe(show_decks, 'value')

export_button = Button(description="Export selected deck", layout=auto_layout(), button_style='primary')
export_button.on_click(export_deck)

VBox([HBox([VBox([file_uploader, export_button]), select_deck]), out_text, out])
